In [1]:
import pandas as pd
import numpy as np
import os
import re
from PIL import Image
import h5py
from sklearn.model_selection import train_test_split
import argparse

In [2]:
import copy
import logging
import torch
import torch.nn as nn
from torchvision import *
import random
import sys
import time
import cv2

In [3]:
img_size = 512

val_output_path = 'val.h5'
#img_dir =  r'ISIC2018_Task1-2_Validation_Input/'
#mask_dir = 'ISIC2018_Task2_Validation_GroundTruth/'
img_dir =  r'ISIC2018_Task1-2_Training_Input/'
mask_dir = 'ISIC2018_Task2_Training_GroundTruth_v3/'


IMG_FILE_REGEX = r'ISIC_(\d+).jpg'
MASK_FILE_REGEX = r'ISIC_(\d+)_attribute_(.*).png'

ISIC2019_FILE_REGEX = r'ISIC_(\d+).jpg'

ATTR_TO_INDEX = {
    'globules' : 0,
    'milia_like_cyst' : 1,
    'negative_network' : 2,
    'pigment_network' : 3,
    'streaks' : 4,
}

In [4]:
Back = '../../../../../594A/Dataset/ISIC challenge 2018 Task 1-2/'

img_dir =  Back + img_dir
mask_dir = Back + mask_dir

#  ISIC2018 Val

In [5]:
df_ISIC2018 = pd.DataFrame(columns=['image', 'globules', 'milia_like_cyst', 
                                    'negative_network', 'pigment_network', 'streaks'])

In [6]:
df_ISIC2018.head()

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks


In [7]:
for img_file in os.listdir( img_dir ):
    if not img_file.endswith('.jpg'): continue
    img_id = str( re.search( IMG_FILE_REGEX, img_file ).group(1) )
    img_name = 'ISIC_' + img_id
    df_append = pd.DataFrame.from_records([{'image': img_name, 'globules': 2, 'milia_like_cyst': 2, 
                                      'negative_network': 2, 'pigment_network': 2, 'streaks': 2, 'pigment_network_pred': 2}])
    
    df_ISIC2018 = pd.concat([df_ISIC2018, df_append], ignore_index=True)

In [8]:
df_ISIC2018

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks,pigment_network_pred
0,ISIC_0008913,2,2,2,2,2,2.0
1,ISIC_0000418,2,2,2,2,2,2.0
2,ISIC_0015614,2,2,2,2,2,2.0
3,ISIC_0000079,2,2,2,2,2,2.0
4,ISIC_0013399,2,2,2,2,2,2.0
...,...,...,...,...,...,...,...
2589,ISIC_0010334,2,2,2,2,2,2.0
2590,ISIC_0010493,2,2,2,2,2,2.0
2591,ISIC_0012813,2,2,2,2,2,2.0
2592,ISIC_0011115,2,2,2,2,2,2.0


## 2. save binary input 

In [9]:
'''
for mask_file in os.listdir( mask_dir ):
    if not mask_file.endswith('.png'): continue
    mask = Image.open( os.path.join( mask_dir, mask_file ) )
    mask = mask.resize( (img_size, img_size) )
    mask = np.array( mask )
    
    assert mask.shape == (img_size, img_size)
    assert mask.max() <= 255
    
    # decide feature exist or not
    if mask.max() > 0:
        binary_label = 1
    else:
        binary_label = 0
    
    m = re.search(MASK_FILE_REGEX, mask_file)
    
    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]
    
    if m.group(2) ==  'globules':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label
    elif m.group(2) ==  'milia_like_cyst':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label
    elif m.group(2) ==  'negative_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = binary_label
    elif m.group(2) ==  'pigment_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('pigment_network')]  = binary_label
    elif m.group(2) ==  'streaks':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('streaks')]  = binary_label
'''

"\nfor mask_file in os.listdir( mask_dir ):\n    if not mask_file.endswith('.png'): continue\n    mask = Image.open( os.path.join( mask_dir, mask_file ) )\n    mask = mask.resize( (img_size, img_size) )\n    mask = np.array( mask )\n    \n    assert mask.shape == (img_size, img_size)\n    assert mask.max() <= 255\n    \n    # decide feature exist or not\n    if mask.max() > 0:\n        binary_label = 1\n    else:\n        binary_label = 0\n    \n    m = re.search(MASK_FILE_REGEX, mask_file)\n    \n    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]\n    \n    if m.group(2) ==  'globules':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label\n    elif m.group(2) ==  'milia_like_cyst':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label\n    elif m.group(2) ==  'negative_network':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = bina

In [10]:
#df_ISIC2018.to_csv('df_ISIC2018_val.csv', index=False)
csv_path = '../../../../../Explainable-NN/Data/val_fold_1.csv'
df_ISIC2018 = pd.read_csv(csv_path)

# Train.h5py

In [11]:
with h5py.File(val_output_path, "w") as f:
    count = len(df_ISIC2018)

    # store imgs to h5 file
    images = f.create_dataset( 'images', (count, 3, img_size, img_size), dtype=np.uint8 )
    image_ids = f.create_dataset( 'image_ids', (count,), dtype=int )
    img_id_to_h5idx = {}
    i = 0
    for idx in range(len(df_ISIC2018)):
        print( f'Processing image {i+1}/{count}' )
        img_file = df_ISIC2018.iloc[idx].image + '.jpg'
        img = Image.open( os.path.join( img_dir, img_file ) )
        img = img.resize( (img_size, img_size) )
        img = np.array( img )
        img = img.transpose(2, 0, 1)
        #(3,512,1024)
        assert img.shape == (3, img_size, img_size)
        assert img.max() <= 255
        images[i] = img
        img_id = int( re.search( IMG_FILE_REGEX, img_file ).group(1) )
        image_ids[i] = img_id
        img_id_to_h5idx[ img_id ] = i
        i += 1
        
    # store masks to h5 file
    i = 0
    masks = f.create_dataset('masks', (count, 5, img_size, img_size), dtype=np.uint8)
    attr_labels = f.create_dataset('labels', (count, 5), dtype=np.uint8)
    for mask_file in os.listdir( mask_dir ):
        if not mask_file.endswith('.png'): continue
        print( f'Processing mask {i+1}/{5*count}' )
        mask = Image.open( os.path.join( mask_dir, mask_file ) )
        mask = mask.resize( (img_size, img_size) )
        mask = np.array( mask )
        assert mask.shape == (img_size, img_size)
        assert mask.max() <= 255
        m = re.search(MASK_FILE_REGEX, mask_file)
        img_id, attr_id = int(m.group(1)), ATTR_TO_INDEX[ m.group(2) ]
        
        
        h5idx = img_id_to_h5idx[ img_id ]
        masks[ h5idx, attr_id ] = mask 
        if mask.max() > 0:
            attr_labels[h5idx, attr_id] = 1
        i += 1      

Processing image 1/519
Processing image 2/519
Processing image 3/519
Processing image 4/519
Processing image 5/519
Processing image 6/519
Processing image 7/519
Processing image 8/519
Processing image 9/519
Processing image 10/519
Processing image 11/519
Processing image 12/519
Processing image 13/519
Processing image 14/519
Processing image 15/519
Processing image 16/519
Processing image 17/519
Processing image 18/519
Processing image 19/519
Processing image 20/519
Processing image 21/519
Processing image 22/519
Processing image 23/519
Processing image 24/519
Processing image 25/519
Processing image 26/519
Processing image 27/519
Processing image 28/519
Processing image 29/519
Processing image 30/519
Processing image 31/519
Processing image 32/519
Processing image 33/519
Processing image 34/519
Processing image 35/519
Processing image 36/519
Processing image 37/519
Processing image 38/519
Processing image 39/519
Processing image 40/519
Processing image 41/519
Processing image 42/519
P

Processing image 334/519
Processing image 335/519
Processing image 336/519
Processing image 337/519
Processing image 338/519
Processing image 339/519
Processing image 340/519
Processing image 341/519
Processing image 342/519
Processing image 343/519
Processing image 344/519
Processing image 345/519
Processing image 346/519
Processing image 347/519
Processing image 348/519
Processing image 349/519
Processing image 350/519
Processing image 351/519
Processing image 352/519
Processing image 353/519
Processing image 354/519
Processing image 355/519
Processing image 356/519
Processing image 357/519
Processing image 358/519
Processing image 359/519
Processing image 360/519
Processing image 361/519
Processing image 362/519
Processing image 363/519
Processing image 364/519
Processing image 365/519
Processing image 366/519
Processing image 367/519
Processing image 368/519
Processing image 369/519
Processing image 370/519
Processing image 371/519
Processing image 372/519
Processing image 373/519


KeyError: 14795